# Imaging inverse problems with adversarial networks

This example shows you how to train various networks using adversarial training for deblurring problems. We demonstrate running training and inference using a conditional GAN (i.e DeblurGAN), CSGM, AmbientGAN and UAIR implemented in the `deepinv` library, and how to simply train your own GAN by using `deepinv.training.AdversarialTrainer`. These examples can also be easily extended to train more complicated GANs such as CycleGAN.

- Kupyn et al., [_DeblurGAN: Blind Motion Deblurring Using Conditional Adversarial Networks_](https://openaccess.thecvf.com/content_cvpr_2018/papers/Kupyn_DeblurGAN_Blind_Motion_CVPR_2018_paper.pdf)
- Bora et al., [_Compressed Sensing using Generative Models_](https://arxiv.org/abs/1703.03208) (CSGM)
- Bora et al., [_AmbientGAN: Generative models from lossy measurements_](https://openreview.net/forum?id=Hy7fDog0b)
- Pajot et al., [_Unsupervised Adversarial Image Reconstruction_](https://openreview.net/forum?id=BJg4Z3RqF7)

Adversarial networks are characterised by the addition of an adversarial loss $\mathcal{L}_\text{adv}$ to the standard reconstruction loss:

$$\mathcal{L}_\text{adv}(x,\hat x;D)=\mathbb{E}_{x\sim p_x}\left[q(D(x))\right]+\mathbb{E}_{\hat x\sim p_{\hat x}}\left[q(1-D(\hat x))\right]$$

where $D(\cdot)$ is the discriminator model, $x$ is the reference image, $\hat x$ is the estimated reconstruction, $q(\cdot)$ is a quality function (e.g $q(x)=x$ for WGAN). Training alternates between generator $f$ and discriminator $D$ in a minimax game. When there are no ground truths (i.e unsupervised), this may be defined on the measurements $y$ instead.

**Conditional GAN** forward pass: 

$$\hat x = f(y)$$

**Conditional GAN** loss: 

$$\mathcal{L}=\mathcal{L}_\text{sup}(\hat x, x)+\mathcal{L}_\text{adv}(\hat x, x;D)$$

where $\mathcal{L}_\text{sup}$ is a supervised loss such as pixel-wise MSE or VGG Perceptual Loss.

**CSGM**/**AmbientGAN** forward pass: 

$$\hat x = f(z),\quad z\sim \mathcal{N}(\mathbf{0},\mathbf{I}_k)$$

**CSGM** loss:

$$\mathcal{L}=\mathcal{L}_\text{adv}(\hat x, x;D)$$

**AmbientGAN** loss (where $A(\cdot)$ is the physics): 

$$\mathcal{L}=\mathcal{L}_\text{adv}(A(\hat x), y;D)$$

**CSGM**/**AmbientGAN** forward pass at eval time:

$$\hat x = f(\hat z)\quad\text{s.t.}\quad\hat z=\operatorname*{argmin}_z \lVert A(f(z))-y\rVert _2^2$$

**UAIR** forward pass:

$$\hat x = f(y)$$

**UAIR** loss: 

$$\mathcal{L}=\mathcal{L}_\text{adv}(\hat y, y;D)+\lVert A(f(\hat y))- \hat y\rVert^2_2,\quad\hat y=A(\hat x)$$

In [1]:
import deepinv as dinv
from deepinv.loss import adversarial
import torch
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor, CenterCrop
from torchvision.datasets.utils import download_and_extract_archive

device = dinv.utils.get_freer_gpu() if torch.cuda.is_available() else "cpu"

Load data and apply some forward degradation to the images. For this example we use the Urban100 dataset resized to 64x64. For simplicity we apply an isotropic Gaussian blur for demonstration, although the original papers deal with inverse problems with unknown blurs.

In [2]:
physics = dinv.physics.Blur(dinv.physics.blur.gaussian_blur(sigma=(5, 5)))

In [3]:
download_and_extract_archive(
    "https://huggingface.co/datasets/eugenesiow/Urban100/resolve/main/data/Urban100_HR.tar.gz?download=true",
    "Urban100",
    filename="Urban100_HR.tar.gz",
    md5="65d9d84a34b72c6f7ca1e26a12df1e4c"
)

Using downloaded and verified file: Urban100\Urban100_HR.tar.gz
Extracting Urban100\Urban100_HR.tar.gz to Urban100


In [4]:
train_dataset, test_dataset = random_split(ImageFolder("Urban100", transform=Compose([ToTensor(), CenterCrop(64)])), (0.8, 0.2))

dataset_path = dinv.datasets.generate_dataset(
    train_dataset= train_dataset,
    test_dataset = test_dataset,
    physics=physics,
    device=device,
    save_dir=f"Urban100",
    )
    
train_dataloader = DataLoader(
    dinv.datasets.HDF5Dataset(dataset_path, train=True), 
    shuffle=True
)
test_dataloader = DataLoader(
    dinv.datasets.HDF5Dataset(dataset_path, train=False),
    shuffle=False
)

Computing train measurement vectors from base dataset...


100%|██████████| 2/2 [00:03<00:00,  1.88s/it]


Computing test measurement vectors from base dataset...


100%|██████████| 5/5 [00:00<00:00, 10.77it/s]

Dataset has been saved in Urban100


Define reconstruction network (i.e conditional generator) and discriminator network to use for adversarial training. For demonstration we use a simple U-Net as the reconstruction network and the discriminator from [PatchGAN](https://arxiv.org/abs/1611.07004), but these can be replaced with any architecture e.g transformers, unrolled etc. Further discriminator models are in `deepinv.models.gan`.

In [5]:
def get_models(model=None, D=None, lr_g=1e-4, lr_d=1e-4):
    if model is None:
        model = dinv.models.UNet(
            in_channels=3, 
            out_channels=3,
            scales=2,
            circular_padding=True,
            batch_norm=False
            )

    if D is None:
        D = dinv.models.PatchGANDiscriminator(
            n_layers=2,
            batch_norm=False
        )

    optimizer = dinv.training.adversarial.AdversarialOptimizer(
        torch.optim.Adam(model.parameters(), lr=lr_g, weight_decay=1e-8),
        torch.optim.Adam(D.parameters(),     lr=lr_d, weight_decay=1e-8),
    )
    scheduler = dinv.training.adversarial.AdversarialScheduler(
        torch.optim.lr_scheduler.StepLR(optimizer.G, step_size=5, gamma=0.9),
        torch.optim.lr_scheduler.StepLR(optimizer.D, step_size=5, gamma=0.9)
    )
    
    return model, D, optimizer, scheduler

### Conditional GAN training

In [6]:
model, D, optimizer, scheduler = get_models()

Construct pixel-wise and adversarial losses as defined above. We use the MSE for the supervised pixel-wise metric for simplicity but this can be easily replaced with a perceptual loss if desired.

In [7]:
loss_g = [
    dinv.loss.SupLoss(metric=torch.nn.MSELoss()),
    adversarial.SupAdversarialGeneratorLoss(device=device)
]
loss_d = adversarial.SupAdversarialDiscriminatorLoss(device=device)

Train the networks using `AdversarialTrainer`

In [8]:
model = dinv.training.AdversarialTrainer(
    model=model,
    D=D,
    physics=physics,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader,
    epochs=3,
    losses=loss_g,
    losses_d=loss_d,
    optimizer=optimizer,
    scheduler=scheduler,
    verbose=True,
    progress_bar=False,
    save_path=None,
    device=device
).train()

The model has 444867 trainable parameters
Eval epoch 0: PSNR=14.884
Train epoch 0: SupLoss=0.023 SupAdversarialGeneratorLoss=0.004 TotalLoss=1.0 PSNR=17.141
Eval epoch 1: PSNR=17.094
Train epoch 1: SupLoss=0.022 SupAdversarialGeneratorLoss=0.004 TotalLoss=1.0 PSNR=17.635
Eval epoch 2: PSNR=17.57
Train epoch 2: SupLoss=0.022 SupAdversarialGeneratorLoss=0.004 TotalLoss=1.0 PSNR=17.787


### UAIR training

In [9]:
model, D, optimizer, scheduler = get_models(lr_g=1e-4, lr_d=4e-4) # learning rates from original paper

Construct losses as defined above

In [10]:
loss_g = adversarial.UAIRGeneratorLoss(device=device)
loss_d = adversarial.UAIRDiscriminatorLoss(device=device)

Train the networks using `AdversarialTrainer`

In [11]:
model = dinv.training.AdversarialTrainer(
    model=model,
    D=D,
    physics=physics,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader,
    epochs=3,
    losses=loss_g,
    losses_d=loss_d,
    optimizer=optimizer,
    scheduler=scheduler,
    verbose=True,
    progress_bar=False,
    save_path=None,
    device=device
).train()

The model has 444867 trainable parameters
Eval epoch 0: PSNR=15.679
Train epoch 0: TotalLoss=1.0 PSNR=16.242
Eval epoch 1: PSNR=15.989
Train epoch 1: TotalLoss=1.0 PSNR=16.176
Eval epoch 2: PSNR=16.112
Train epoch 2: TotalLoss=1.0 PSNR=16.158


### CSGM / AmbientGAN training

In [12]:
model = dinv.models.CSGMGenerator(dinv.models.DCGANGenerator(nz=100, ngf=32), inf_tol=1e-2)
D = dinv.models.DCGANDiscriminator(ndf=32)
_, _, optimizer, scheduler = get_models(model=model, D=D, lr_g=2e-4, lr_d=2e-4) # learning rates from original paper

Construct losses as defined above. We are free to choose between supervised and unsupervised adversarial losses, where supervised gives CSGM and unsupervised gives AmbientGAN.

In [13]:
loss_g = adversarial.SupAdversarialGeneratorLoss(device=device)
loss_d = adversarial.SupAdversarialDiscriminatorLoss(device=device)

Train the networks using `AdversarialTrainer`. Since inference is very slow for CSGM/AmbientGAN as it requires an optimisation, we only do one evaluation at the end. Note the train PSNR is meaningless as this generative model is trained on random latents.

In [14]:
trainer = dinv.training.AdversarialTrainer(
    model=model,
    D=D,
    physics=physics,
    train_dataloader=train_dataloader,
    epochs=3,
    losses=loss_g,
    losses_d=loss_d,
    optimizer=optimizer,
    scheduler=scheduler,
    verbose=True,
    progress_bar=False,
    save_path=None,
    device=device
)
model = trainer.train()

The model has 1100224 trainable parameters
Train epoch 0: TotalLoss=1.0 PSNR=2.244
Train epoch 1: TotalLoss=1.0 PSNR=2.672
Train epoch 2: TotalLoss=1.0 PSNR=2.942


Run evaluation of generative model by running test-time optimisation using test measurements. Note that we do not get great results as CSGM / AmbientGAN relies on large datasets of diverse samples, and we run the optimisation to a relatively high tolerance for speed.

In [17]:
psnr = trainer.test(test_dataloader)[0]
print("Test PSNR", psnr)

Test PSNR: No learning rec.: 15.945+-2.634 | Model: 3.521+-0.816. 
Test PSNR 3.521284806728363
